In [1]:
from glob import glob
import numpy as np
data_path = "/Users/soltan/Programs/kaggle/raw_data/train/train/*"

import librosa
def extract_features(file_name):
    n_fft = 2048
    hop_length = 512
    signal, sr = librosa.load(file_name, sr=None)
    mfcc = librosa.feature.mfcc(signal, n_fft=n_fft, hop_length=hop_length, n_mfcc=13).flatten()
    return mfcc

In [2]:
from tqdm import tqdm
data_dir = np.array(glob(data_path))
features, labels = [], []
for file in tqdm(data_dir):
    file_name = file.split("/")[-1]
    file_name = file.split(".")[0]
    name, label = file_name.split("-")[0], file_name.split("-")[1]
    features.append(extract_features(file))
    labels.append(label)



100%|██████████| 9000/9000 [00:44<00:00, 201.77it/s]


In [3]:
from sklearn.model_selection import train_test_split
inputs_train, inputs_test, targets_train, targets_test = train_test_split(features, labels, test_size=0.2)

In [15]:
from sklearn import svm

clf = svm.SVC()
clf.fit(inputs_train, targets_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
from sklearn import metrics
predictions = clf.predict(inputs_test)
print("Accuracy:",metrics.accuracy_score(targets_test, predictions))


Accuracy: 0.6211111111111111


In [18]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=1000, n_jobs=8)
grid.fit(inputs_train,targets_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:  2.6min
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:  2.6min
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:  2.6min
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:  2.7min
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:  2.7min
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:  3.2min
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  3.5min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  3.5min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  5.9min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  5.9min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  5.9min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  6.4min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  6.5min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  6.5min
[Parallel

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=8,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1000)

In [19]:
grid_predictions = grid.predict(inputs_test)
print("Grid Accuracy:",metrics.accuracy_score(targets_test, grid_predictions))

Grid Accuracy: 0.6327777777777778


In [23]:
test_path = "/Users/soltan/Programs/kaggle/raw_data/test/test/*"
from tqdm import tqdm
test_dir = np.array(glob(test_path))

fout = open("../submission.txt", "w")

for file in tqdm(test_dir):
    name = file.split("/")[-1]
    ft = extract_features(file)
    pred = grid.predict([ft])[0]
    fout.write("{},{}\n".format(name, pred))

fout.close()

100%|██████████| 3000/3000 [00:47<00:00, 63.47it/s]
